# This laptop was created to demonstrate the operation of an adapter for CodeLlama, designed to solve the problem of improving the readability of decompiled C code

# 1) Install the necessary libraries

In [1]:
!pip install accelerate
!pip install bitsandbytes
!pip install peft
!pip install transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 1.8 MB/s eta 0:00:00a 0:00:01


# 2) Import block

In [3]:
from datetime import datetime
import os
import sys
import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training,
    set_peft_model_state_dict,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, LlamaTokenizer, LlamaForCausalLM, BitsAndBytesConfig

2024-05-21 19:30:34.036878: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 19:30:34.036999: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 19:30:34.191210: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# 3) Set device, basic model's tokenizer

In [4]:
# Set which device we will use (GPU or CPU).
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# The basic model
BASE_MODEL = "codellama/CodeLlama-7b-hf"
# Initializing the tokenizer
tokenizer = LlamaTokenizer.from_pretrained(BASE_MODEL)
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/749 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'CodeLlamaTokenizer'. 
The class this function is called from is 'LlamaTokenizer'.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


# 4) Upload our eval dataset with code examples (optional)

In [5]:
from datasets import load_dataset
eval_dataset = load_dataset('csv', data_files='/kaggle/input/9000hr/9000hr (1).csv', split='train[0%:1%]')
print(len(eval_dataset))
a1 = '//FormAI DATASET v1.0 Category: Educational ; Style: Donald Knuth\r\n#include <stdio.h>\r\n\r\nint main() {\r\n  int n = 7;\r\n  int arr[n][n];\r\n\r\n  // filling the matrix\r\n  for(int i = 0; i < n; i++) {\r\n    for(int j = 0; j < n; j++) {\r\n      if(i == 0) {\r\n        arr[i][j] = j + 1;\r\n      } else if(j == 0) {\r\n        arr[i][j] = i + 1;\r\n      } else {\r\n        arr[i][j] = arr[i-1][j] + arr[i][j-1];\r\n      }\r\n    }\r\n  }\r\n\r\n  // printing the matrix\r\n  printf("The Pascal\'s Triangle of order %d is:\\n", n);\r\n  for(int i = 0; i < n; i++) {\r\n    for(int j = 0; j < n; j++) {\r\n      printf("%d\\t", arr[i][j]);\r\n    }\r\n    printf("\\n");\r\n  }\r\n\r\n  return 0;\r\n}'
a2 = '/* This file was generated by the Hex-Rays decompiler version 8.3.0.230608.\r\n   Copyright (c) 2007-2021 Hex-Rays <info@hex-rays.com>\r\n\r\n   Detected compiler: Visual C++\r\n*/\r\n\r\n#include <windows.h>\r\n#include <defs.h>\r\n\r\n\r\n//-------------------------------------------------------------------------\r\n// Function declarations\r\n\r\nint printf(const char *const Format, ...);\r\nint __fastcall main(int argc, const char **argv, const char **envp);\r\n__int64 __fastcall _main(_QWORD, _QWORD, _QWORD); // weak\r\n\r\n\r\n//----- (0000000140001591) ----------------------------------------------------\r\nint __fastcall main(int argc, const char **argv, const char **envp)\r\n{\r\n  void *v3; // rsp\r\n  int v4; // ecx\r\n  __int64 v5; // rdx\r\n  __int64 v7; // [rsp+0h] [rbp-80h] BYREF\r\n  char v8; // [rsp+20h] [rbp-60h] BYREF\r\n  __int64 *v9; // [rsp+28h] [rbp-58h]\r\n  __int64 v10; // [rsp+30h] [rbp-50h]\r\n  __int64 v11; // [rsp+38h] [rbp-48h]\r\n  __int64 v12; // [rsp+40h] [rbp-40h]\r\n  __int64 v13; // [rsp+48h] [rbp-38h]\r\n  char *v14; // [rsp+50h] [rbp-30h]\r\n  __int64 v15; // [rsp+58h] [rbp-28h]\r\n  __int64 v16; // [rsp+60h] [rbp-20h]\r\n  int v17; // [rsp+6Ch] [rbp-14h]\r\n  int m; // [rsp+70h] [rbp-10h]\r\n  int k; // [rsp+74h] [rbp-Ch]\r\n  int j; // [rsp+78h] [rbp-8h]\r\n  int i; // [rsp+7Ch] [rbp-4h]\r\n\r\n  _main(argc, argv, envp);\r\n  v9 = &v7;\r\n  v17 = 7;\r\n  v16 = 6i64;\r\n  v10 = 7i64;\r\n  v11 = 0i64;\r\n  v15 = 6i64;\r\n  v12 = 7i64;\r\n  v13 = 0i64;\r\n  v3 = alloca(208i64);\r\n  v14 = &v8;\r\n  for ( i = 0; i < v17; ++i )\r\n  {\r\n    for ( j = 0; j < v17; ++j )\r\n    {\r\n      if ( i )\r\n      {\r\n        if ( j )\r\n        {\r\n          v4 = *(_DWORD *)&v14[28 * i - 28 + 4 * j] + *(_DWORD *)&v14[28 * i - 4 + 4 * j];\r\n          v5 = j + 7i64 * i;\r\n        }\r\n        else\r\n        {\r\n          v4 = i + 1;\r\n          v5 = 7i64 * i;\r\n        }\r\n        *(_DWORD *)&v14[4 * v5] = v4;\r\n      }\r\n      else\r\n      {\r\n        *(_DWORD *)&v14[4 * j] = j + 1;\r\n      }\r\n    }\r\n  }\r\n  printf("The Pascal\'s Triangle of order %d is:\\n", (unsigned int)v17);\r\n  for ( k = 0; k < v17; ++k )\r\n  {\r\n    for ( m = 0; m < v17; ++m )\r\n      printf("%d\\t", *(unsigned int *)&v14[28 * k + 4 * m]);\r\n    printf("\\n");\r\n  }\r\n  return 0;\r\n}\r\n// 140001900: using guessed type __int64 __fastcall _main(_QWORD, _QWORD, _QWORD);\r\n\r\n// nfuncs=137 queued=1 decompiled=1 lumina nreq=0 worse=0 better=0\r\n// ALL OK, 1 function(s) have been successfully decompiled\r\n'

Generating train split: 0 examples [00:00, ? examples/s]

94


In [15]:
# Example of decompiler output:
print(eval_dataset[0][a2])

/* This file was generated by the Hex-Rays decompiler version 8.3.0.230608.
   Copyright (c) 2007-2021 Hex-Rays <info@hex-rays.com>

   Detected compiler: Visual C++
*/

#include <windows.h>
#include <defs.h>


//-------------------------------------------------------------------------
// Function declarations

int scanf(const char *const Format, ...);
int printf(const char *const Format, ...);
int __fastcall main(int argc, const char **argv, const char **envp);
__int64 __fastcall _main(_QWORD, _QWORD, _QWORD); // weak


//----- (00000001400015E2) ----------------------------------------------------
int __fastcall main(int argc, const char **argv, const char **envp)
{
  void *v3; // rsp
  int v5; // [rsp+20h] [rbp-20h] BYREF
  int v6; // [rsp+24h] [rbp-1Ch] BYREF
  int *v7; // [rsp+28h] [rbp-18h]
  __int64 v8; // [rsp+30h] [rbp-10h]
  int j; // [rsp+38h] [rbp-8h]
  int i; // [rsp+3Ch] [rbp-4h]

  _main(argc, argv, envp);
  printf("Enter the number of terms you want to generate in the F

# 5) Download the model together with our checkpoint

In [8]:
model = LlamaForCausalLM.from_pretrained(
    '/kaggle/input/explain-2400', #checkpoint folder
    torch_dtype=torch.float16,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type='nf4',
    )
)

config.json:   0%|          | 0.00/637 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [9]:
# Architecture of the model with an adapter
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32016, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=4096, out_features=1, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=1, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)

In [11]:
# Additional preparation of the model for use
from peft import PeftModel
model = PeftModel.from_pretrained(model, '/kaggle/input/explain-2400')
model.config.pad_token_id = tokenizer.pad_token_id = 0 
model.config.bos_token_id = 1
model.config.eos_token_id = 2

# 6) Evaluation

In [13]:
eval_prompt = """You are a powerful decompiler model. Your job is to convert the С code decompiled using Hex-Rays decompiler into a more human-readable form. That is, you should change the names of variables and functions and delete unnecessary parts of the code so that it looks more like the source code of a C program. You are given a С code decompiled using Hex-Rays decompiler.

You must output the source code of a C program.

### Your input:
/* This file was generated by the Hex-Rays decompiler version 8.3.0.230608.
   Copyright (c) 2007-2021 Hex-Rays <info@hex-rays.com>

   Detected compiler: Visual C++
*/

#include <windows.h>
#include <defs.h>


//-------------------------------------------------------------------------
// Function declarations

int scanf(const char *const Format, ...);
int printf(const char *const Format, ...);
int __fastcall main(int argc, const char **argv, const char **envp);
__int64 __fastcall _main(_QWORD, _QWORD, _QWORD); // weak


//----- (00000001400015E2) ----------------------------------------------------
int __fastcall main(int argc, const char **argv, const char **envp)
{
  void *v3; // rsp
  int v5; // [rsp+20h] [rbp-20h] BYREF
  int v6; // [rsp+24h] [rbp-1Ch] BYREF
  int *v7; // [rsp+28h] [rbp-18h]
  __int64 v8; // [rsp+30h] [rbp-10h]
  int j; // [rsp+38h] [rbp-8h]
  int i; // [rsp+3Ch] [rbp-4h]

  _main(argc, argv, envp);
  printf("Enter the number of terms you want to generate in the Fibonacci Sequence: ");
  scanf("%d", &v6);
  v8 = v6 - 1i64;
  v3 = alloca(16 * ((unsigned __int64)(4i64 * v6 + 15) >> 4));
  v7 = &v5;
  v5 = 0;
  v6 = 1;
  i = 2;
  printf("\nFibonacci Sequence for %d terms is: ", 1i64);
  for ( i = 0; i < v6; ++i )
    printf("%d ", (unsigned int)v7[i]);
  printf("\n\nVisualizing the Fibonacci Sequence:\n");
  for ( i = 0; i < v6; ++i )
  {
    for ( j = 0; j < v7[i]; ++j )
      printf("* ");
    printf("\n");
  }
  printf("\n");
  return 0;
}
// 1400016CE: conditional instruction was optimized away because %var_34.4==2
// 140001850: using guessed type __int64 __fastcall _main(_QWORD, _QWORD, _QWORD);

// nfuncs=176 queued=1 decompiled=1 lumina nreq=0 worse=0 better=0
// ALL OK, 1 function(s) have been successfully decompiled

### The original C program:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=500)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a powerful decompiler model. Your job is to convert the С code decompiled using Hex-Rays decompiler into a more human-readable form. That is, you should change the names of variables and functions and delete unnecessary parts of the code so that it looks more like the source code of a C program. You are given a С code decompiled using Hex-Rays decompiler.

You must output the source code of a C program.

### Your input:
/* This file was generated by the Hex-Rays decompiler version 8.3.0.230608.
   Copyright (c) 2007-2021 Hex-Rays <info@hex-rays.com>

   Detected compiler: Visual C++
*/

#include <windows.h>
#include <defs.h>


//-------------------------------------------------------------------------
// Function declarations

int scanf(const char *const Format, ...);
int printf(const char *const Format, ...);
int __fastcall main(int argc, const char **argv, const char **envp);
__int64 __fastcall _main(_QWORD, _QWORD, _QWORD); // weak


//----- (00000001400015E2) --------------